In [ ]:
import pandas as pd
import requests, zipfile, io
from datetime import datetime, timedelta

# Caricamento della lista dei file GDELT [cite: 4, 5]
master_url = "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"
files = pd.read_csv(master_url, sep=" ", header=None, low_memory=False) # Usa spazio come separatore
files = files[[2]] # Seleziona la colonna degli URL
files.columns = ["url"] # Rinomina la colonna

def count_mentions_total_day(day_string, keyword):
    """Conta le menzioni in TUTTI i file (15min e translation) di un giorno [cite: 33]"""
    # Filtra tutti i file GKG del giorno, inclusi quelli tradotti
    daily_urls = files[
        files["url"].str.contains(day_string) &
        files["url"].str.contains("gkg.csv.zip")
    ]["url"].tolist()

    total_count = 0
    print(f"Analizzando {day_string} ({len(daily_urls)} file)...")

    for url in daily_urls:
        try:
            r = requests.get(url, timeout=10) # Download del file zip
            z = zipfile.ZipFile(io.BytesIO(r.content))
            fname = z.namelist()[0] # Estrazione nome file
            # Legge il CSV e conta le occorrenze della keyword nella colonna 14
            df = pd.read_csv(z.open(fname), sep="\t", header=None, encoding='latin-1', usecols=[14])
            total_count += df[14].str.contains(keyword, case=False, na=False).sum()
        except:
            continue # Salta eventuali file corrotti o errori di rete

    return total_count

def normalize_to_percentage(df, value_col):
    """Normalizza i valori in un indice da 0 a 100 [cite: 121]"""
    max_value = df[value_col].max() # Trova il massimo nel periodo
    if max_value > 0:
        df["gdelt_index_pct"] = (df[value_col] / max_value) * 100 # Calcolo percentuale
    else:
        df["gdelt_index_pct"] = 0
    return df

**NETFLIX**

In [ ]:
results_netflix = []
start_n = datetime(2022, 4, 19) # Data inizio richiesta
end_n = datetime(2022, 5, 3)    # Data fine richiesta
current = start_n

while current <= end_n:
    day_str = current.strftime("%Y%m%d") # Formattazione data per GDELT
    count = count_mentions_total_day(day_str, "Netflix")
    results_netflix.append({"day": current, "gdelt_volume": count})
    current += timedelta(days=1)

df_netflix = pd.DataFrame(results_netflix)
df_netflix = normalize_to_percentage(df_netflix, "gdelt_volume") # Normalizzazione
print(df_netflix)

Analizzando 20220419 (96 file)...
Analizzando 20220420 (96 file)...
Analizzando 20220421 (96 file)...
Analizzando 20220422 (96 file)...
Analizzando 20220423 (96 file)...
Analizzando 20220424 (96 file)...
Analizzando 20220425 (96 file)...
Analizzando 20220426 (96 file)...
Analizzando 20220427 (96 file)...
Analizzando 20220428 (96 file)...
Analizzando 20220429 (96 file)...
Analizzando 20220430 (96 file)...
Analizzando 20220501 (96 file)...
Analizzando 20220502 (96 file)...
Analizzando 20220503 (96 file)...
          day  gdelt_volume  gdelt_index_pct
0  2022-04-19          1400        38.620690
1  2022-04-20          3625       100.000000
2  2022-04-21          1855        51.172414
3  2022-04-22          1340        36.965517
4  2022-04-23           848        23.393103
5  2022-04-24           560        15.448276
6  2022-04-25           883        24.358621
7  2022-04-26           826        22.786207
8  2022-04-27           971        26.786207
9  2022-04-28          1180        32.55

**SVB**

In [ ]:
results_svb = []
start_s = datetime(2023, 3, 10) # Data inizio richiesta
end_s = datetime(2023, 3, 17)   # Data fine richiesta
current = start_s

while current <= end_s:
    day_str = current.strftime("%Y%m%d")
    count = count_mentions_total_day(day_str, "Silicon Valley Bank")
    results_svb.append({"day": current, "gdelt_volume": count})
    current += timedelta(days=1)

df_svb = pd.DataFrame(results_svb)
df_svb = normalize_to_percentage(df_svb, "gdelt_volume") # Normalizzazione
print(df_svb)

Analizzando 20230310 (96 file)...
Analizzando 20230311 (94 file)...
Analizzando 20230312 (95 file)...
Analizzando 20230313 (96 file)...
Analizzando 20230314 (94 file)...
Analizzando 20230315 (96 file)...
Analizzando 20230316 (95 file)...
Analizzando 20230317 (96 file)...
         day  gdelt_volume  gdelt_index_pct
0 2023-03-10          1208        19.386936
1 2023-03-11          1935        31.054405
2 2023-03-12          2614        41.951533
3 2023-03-13          6231       100.000000
4 2023-03-14          5228        83.903065
5 2023-03-15          4160        66.762959
6 2023-03-16          3976        63.809982
7 2023-03-17          3417        54.838710


**CELLE DI VISUALIZZAZIONE DATI**

In [ ]:
import pandas as pd
from IPython.display import display, HTML

def format_results(df, title):
    print(f"\n{'='*40}")
    print(f"  {title}")
    print(f"{'='*40}")

    # Formattazione per la stampa: data pulita senza orario
    display_df = df.copy()
    display_df['day'] = display_df['day'].dt.strftime('%Y-%m-%d')

    # Rinomina colonne per chiarezza
    display_df.columns = ["Data", "Volume Menzioni (Ogni 15 minuti un file per 24h, totale 96 file)", "Indice di Attenzione (0-100)"]

    # Stampa la tabella
    display(display_df)

    # Calcolo statistiche rapide
    total_mentions = df['gdelt_volume'].sum()
    peak_row = df.loc[df['gdelt_volume'].idxmax()]

    print(f"RIEPILOGO STATISTICO:")
    print(f"- Volume totale nel periodo: {total_mentions} menzioni")
    print(f"- Giorno di massima attenzione: {peak_row['day'].strftime('%Y-%m-%d')} (Indice: 100)")
    print(f"{'-'*40}\n")

# 1. Visualizza dati Netflix
if 'df_netflix' in locals():
    format_results(df_netflix, "ANALISI MEDIA: NETFLIX (APRILE 2022)")
else:
    print("Dati Netflix non trovati. Esegui prima la cella di analisi.")




  ANALISI MEDIA: NETFLIX (APRILE 2022)


,Data,"Volume Menzioni (Ogni 15 minuti un file per 24h, totale 96 file)",Indice di Attenzione (0-100)
0,2022-04-19,1400,38.620690
1,2022-04-20,3625,100.000000
2,2022-04-21,1855,51.172414
3,2022-04-22,1340,36.965517
4,2022-04-23,848,23.393103
5,2022-04-24,560,15.448276
6,2022-04-25,883,24.358621
7,2022-04-26,826,22.786207
8,2022-04-27,971,26.786207
9,2022-04-28,1180,32.551724


RIEPILOGO STATISTICO:
- Volume totale nel periodo: 18507 menzioni
- Giorno di massima attenzione: 2022-04-20 (Indice: 100)
----------------------------------------



In [ ]:
# CELLA DI VISUALIZZAZIONE DATI

import pandas as pd
from IPython.display import display, HTML

def format_results(df, title):
    print(f"\n{'='*40}")
    print(f"  {title}")
    print(f"{'='*40}")

    # Formattazione per la stampa: data pulita senza orario
    display_df = df.copy()
    display_df['day'] = display_df['day'].dt.strftime('%Y-%m-%d')

    # Rinomina colonne per chiarezza
    display_df.columns = ["Data", "Volume Menzioni (Totale 96 file)", "Indice di Attenzione (0-100)"]

    # Stampa la tabella
    display(display_df)

    # Calcolo statistiche rapide
    total_mentions = df['gdelt_volume'].sum()
    peak_row = df.loc[df['gdelt_volume'].idxmax()]

    print(f"RIEPILOGO STATISTICO:")
    print(f"- Volume totale nel periodo: {total_mentions} menzioni")
    print(f"- Giorno di massima attenzione: {peak_row['day'].strftime('%Y-%m-%d')} (Indice: 100)")
    print(f"{'-'*40}\n")

# 2. Visualizza dati SVB
if 'df_svb' in locals():
    format_results(df_svb, "ANALISI MEDIA: SILICON VALLEY BANK (MARZO 2023)")
else:
    print("Dati SVB non trovati. Esegui prima la cella di analisi.")


  ANALISI MEDIA: SILICON VALLEY BANK (MARZO 2023)


,Data,Volume Menzioni (Totale 96 file),Indice di Attenzione (0-100)
0,2023-03-10,1208,19.386936
1,2023-03-11,1935,31.054405
2,2023-03-12,2614,41.951533
3,2023-03-13,6231,100.000000
4,2023-03-14,5228,83.903065
5,2023-03-15,4160,66.762959
6,2023-03-16,3976,63.809982
7,2023-03-17,3417,54.838710


RIEPILOGO STATISTICO:
- Volume totale nel periodo: 28769 menzioni
- Giorno di massima attenzione: 2023-03-13 (Indice: 100)
----------------------------------------



**SALVATAGGIO IN CSV.**

In [ ]:
# Salvataggio dati Netflix
if 'df_netflix' in locals():
    df_netflix.to_csv('analisi_gdelt_netflix.csv', index=False, sep=';')
    print("File 'analisi_gdelt_netflix.csv' creato con successo.")
else:
    print("Attenzione: df_netflix non trovato. Esegui prima l'analisi.")

# Salvataggio dati SVB
if 'df_svb' in locals():
    df_svb.to_csv('analisi_gdelt_svb.csv', index=False, sep=';')
    print("File 'analisi_gdelt_svb.csv' creato con successo.")
else:
    print("Attenzione: df_svb non trovato. Esegui prima l'analisi.")

File 'analisi_gdelt_netflix.csv' creato con successo.
File 'analisi_gdelt_svb.csv' creato con successo.
